

---
Fine-Tunning do modelo BerTimabau para fazer a classificação de Aspectos ("neuralmind/bert-base-portuguese-cased")


In [ ]:
# =============================
# CÉLULA 1: Instale dependências no Colab
# =============================
%pip install --upgrade transformers datasets --quiet
%pip install transformers datasets scikit-learn --quiet


In [ ]:

# =============================
# CÉLULA 2: Upload ou Google Drive
# =============================
# Para upload manual:
# from google.colab import files
# uploaded = files.upload()  # Faça upload do fine_tunning.csv

# Para Google Drive:
from google.colab import drive
drive.mount('/content/drive')
# csv_path = "/content/drive/MyDrive/sua_pasta/fine_tunning.csv"
csv_path = "/content/drive/MyDrive/TCC/FT_BT_ClassificaAspectos/Aspect_Classification_fine_tunning.csv"  # Ajuste se usar o Drive


In [ ]:
# =============================
# CÉLULA 3: Fine-tuning para classificação binária de aspectos
# =============================
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from datasets import Dataset

model_checkpoint = "neuralmind/bert-base-portuguese-cased"

# Checa se está usando GPU
print("CUDA disponível:", torch.cuda.is_available())

# Carrega o CSV
csv_path = "Aspect_Classification_fine_tunning.csv"
df = pd.read_csv(csv_path, quotechar='"')

# Filtra apenas os labels válidos (0 ou 1)
df = df[df["label"].isin([0, 1])]

# Junta feedback e aspecto para contexto
texts = df["feedback"] + " [ASPECTO] " + df["aspecto"]
labels = df["label"].tolist()

# Split train/test
train_texts, val_texts, train_labels, val_labels = train_test_split(
    texts.tolist(), labels, test_size=0.1, random_state=42, stratify=labels
)

tokenizer = BertTokenizer.from_pretrained(model_checkpoint)

def tokenize(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=128)

train_dataset = Dataset.from_dict({"text": train_texts, "label": train_labels})
val_dataset = Dataset.from_dict({"text": val_texts, "label": val_labels})

train_dataset = train_dataset.map(tokenize, batched=True)
val_dataset = val_dataset.map(tokenize, batched=True)

model = BertForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2)

training_args = TrainingArguments(
    output_dir="./bertimbau-ft-results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    save_total_limit=2,
)

def compute_metrics(eval_pred):
    from sklearn.metrics import accuracy_score, f1_score
    logits, labels = eval_pred
    preds = logits.argmax(axis=-1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average="macro")
    return {"accuracy": acc, "f1": f1}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

# =============================
# CÉLULA FINAL: Treinamento e salvamento do modelo
# =============================
trainer.train()
trainer.save_model("./FT_BT_ClassificaAspectos")
tokenizer.save_pretrained("./FT_BT_ClassificaAspectos")
print("Fine-tuning concluído e modelo + tokenizer salvos em ./FT_BT_ClassificaAspectos")

CUDA disponível: True


Map:   0%|          | 0/1215 [00:00<?, ? examples/s]

Map:   0%|          | 0/135 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<IPython.core.display.Javascript object>

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: eduardo-ferrarini (eduardo-ferrarini-none) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Fine-tuning concluído e modelo salvo em ./bertimbau-ft-TCC
